In [4]:
import os
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pymorphy2
import seaborn as sns
sns.set_style("whitegrid")
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from string import punctuation

# HW1:  Сравнение стилей текстов
### Выполнили:  Булгаков Дмитрий, Тефикова Алие
### Группа ИАД-2

# 1. Загрузка данных

Составьте самостоятельно как минимум две коллекции
текстов разных стилей (например, коллекция текстов в публицистическом
стиле и коллекция текстов в научном стиле). Коллекции текстов
должны быть достаточно большие (порядка 5000 токенов). Посчитайте
количество токенов и типов в каждой коллекции.

### 1.1 Получение  данных из файла

In [5]:
def remove_control_characters(text_string):
    return ''.join(filter(None, text_string.splitlines()))

### 1.1.1 Загрузка художественных текстов (Портрет Дориана Грея,  Оскар Уайльд)

«Портре́т Дориана Гре́я» (англ. The Picture of Dorian Gray) — единственный опубликованный роман Оскара Уайльда. В жанровом отношении представляет смесь романа воспитания с моральной притчей. Существует в двух версиях — в 13 главах (1890 года) и в 20 главах (1891 года). Стал самым успешным произведением Уайльда, более 30 раз экранизировался.

<b>Link:<b> http://lib.ru/WILDE/doriangray.txt

In [6]:
fiction = open('data/dorian_gray.txt', encoding='utf-8').read()
fiction = remove_control_characters(fiction)

### 1.1.2 Загрузка  публицистических текстов (статьи lenta.ru) 

Lenta.ru — одно из ведущих российских новостных интернет-изданий, основанное в 1999 году Антоном Носиком при содействии Фонда эффективной политики. Работает круглосуточно, освещая мировые и внутрироссийские новости.

<b>Link<b>: http://lenta.ru

In [7]:
journalistic = open('data/lentaru.txt', encoding='utf-8').read()
journalistic = remove_control_characters(journalistic)

### 1.1.3 Загрузка  научных текстов (Молодежная Наука 2016)

Материалы Всероссийской научно-практической конференции молодых ученых, аспирантов и студентов, посвященной 150-летию со дня рождения профессора В.Н. Варгина (Пермь, 14-18 марта 2016 года)

<b>Link<b>: http://pgsha.ru/web/science/scientificarticles/

In [8]:
scientific = open('data/perm_conf.txt', encoding='utf-8').read()
scientific = remove_control_characters(scientific)

### 1.1.4 Загрузка  текстов  разговорного стиля (корпус составленный на базе Twitter )

В качестве источника текстов была выбрана платформа микроблогинга Twitter. Современные поисковые системы и имеющиеся в открытом доступе инструменты по сбору текстовых отзывов не позволяют собирать актуальные отзывы и оперативно работать с данными. В связи с этим на основе программного интерефейса API twitter  был разработан программный инструмент для извлечения отзывов об интересующих товарах, услугах,  событиях,  персонах из микроблоггинг-платформы twitter,  который позволяет учитывать время публикации сообщения и авторитетность автора сообщения. Этот инструмент использовался для сбора неразмеченного корпуса. В корпусе содержится более 15 миллионов записей за время с конца ноября 2013 года до конца февраля 2014 года.

<b>Link:<b> http://study.mokoron.com

In [9]:
conversational = open('data/twitter.txt', encoding='utf-8').read()
conversational = remove_control_characters(conversational)

### 1.2 Подсчет токенов и типов

In [10]:
exclude_symbols = set(punctuation + '0123456789'+u'–—'+u'«»'+u'“')

In [11]:
def tokenize(text, exlude_symb):
    text = text.lower()
    text_merged = ''.join(ch for ch in text if ch not in exlude_symb)
    text_tokens = WhitespaceTokenizer().tokenize(text_merged.lower())
    return text_tokens

In [12]:
def print_results(tokens):
    print('N of tokens: ', len(tokens))
    types = nltk.FreqDist(tokens)
    print('N of types:', len(types))

### 1.2.1 Токены и типы для  художественного стиля

In [13]:
fiction_tokens = tokenize(fiction, exclude_symbols)
print_results(fiction_tokens)

N of tokens:  60528
N of types: 18236


### 1.2.2 Токены и типы для публицистического стиля

In [14]:
journalistic_tokens = tokenize(journalistic, exclude_symbols)
print_results(journalistic_tokens)

N of tokens:  13699
N of types: 6015


### 1.2.3 Токены и типы для  научного стиля

In [15]:
scientific_tokens = tokenize(scientific, exclude_symbols)
print_results(scientific_tokens)

N of tokens:  265376
N of types: 57790


### 1.2.4 Токены и типы для  разговорного стиля

In [16]:
conversational_tokens = tokenize(conversational, exclude_symbols)
print_results(conversational_tokens)

N of tokens:  255367
N of types: 56878


### 2. Подсчет частей речи

Используя любой морфологический процессор, который вам нравится (pymorphy2, mystem), определите к какой части речиотносятся слова из каждой коллекции текстов. При помощи nltk.FreqDist() составьте частотные словари: часть речи – количество слов, к ней относящихся.

#### fiction

In [61]:
morph = pymorphy2.MorphAnalyzer()
lemmata = nltk.FreqDist()

f_types = nltk.FreqDist(fiction_tokens)
for t in c_types:
    try:
        l = morph.parse(t)[0].normal_form
        if l in lemmata:
            lemmata[l] += f_types[t]
        else:
            lemmata[l] = f_types[t]
    except IndexError:
        if t in lemmata:
            lemmata[t] += f_types[t]
        else:
            lemmata[t] = f_types[t]
print('N of lemmata:', len(lemmata))
for i in lemmata.most_common(10):
    print(i[0], i[1])

N of lemmata: 40435
и 1914
он 1773
в 1585
я 1477
не 1211
вы 913
что 904
быть 729
на 725
с 689


In [64]:
fiction_stops = stopwords.words('russian') + [u'это']

In [65]:
lemmata_no_sw = nltk.FreqDist()
for l in lemmata:
    if not l in fiction_stops:
        lemmata_no_sw[l] = lemmata[l]
for i in lemmata_no_sw.most_common(20):
    print(i[0], i[1])

весь 451
человек 296
который 282
свой 277
генри 243
жизнь 225
сказать 219
мочь 196
гарри 165
ещё 152
греть 141
знать 134
хотеть 130
говорить 125
рука 124
глаз 122
очень 117
большой 110
стать 107
портрет 101


#### conversational

In [66]:
morph = pymorphy2.MorphAnalyzer()
lemmata = nltk.FreqDist()

c_types = nltk.FreqDist(conversational_tokens)
for t in c_types:
    try:
        l = morph.parse(t)[0].normal_form
        if l in lemmata:
            lemmata[l] += c_types[t]
        else:
            lemmata[l] = c_types[t]
    except IndexError:
        if t in lemmata:
            lemmata[t] += c_types[t]
        else:
            lemmata[t] = c_types[t]
print('N of lemmata:', len(lemmata))
for i in lemmata.most_common(10):
    print(i[0], i[1])

N of lemmata: 40435
я 10214
не 7760
в 6501
и 6376
на 4281
что 3477
с 3452
а 3077
весь 3017
ты 2850


In [73]:
conversational_stops = stopwords.words('russian')+[u'это'+u'оо'+u'изз'+u'б']

In [82]:
lemmata_no_sw = nltk.FreqDist()
for l in lemmata:
    if not l in conversational_stops:
        lemmata_no_sw[l] = lemmata[l]
for i in lemmata_no_sw.most_common(10):
    print(i[0], i[1])

весь 3017
это 2333
ещё 1172
хотеть 1125
мочь 916
день 855
сегодня 790
очень 733
просто 669
свой 663


#### scientific

In [47]:
scientific_stops = stopwords.words('russian')  + [u'№', u'это', u'гсха']

In [34]:
morph = pymorphy2.MorphAnalyzer()
lemmata = nltk.FreqDist()

s_types = nltk.FreqDist(scientific_tokens)
for t in s_types:
    try:
        l = morph.parse(t)[0].normal_form
        if l in lemmata:
            lemmata[l] += s_types[t]
        else:
            lemmata[l] = s_types[t]
    except IndexError:
        if t in lemmata:
            lemmata[t] += s_types[t]
        else:
            lemmata[t] = s_types[t]
print('N of lemmata:', len(lemmata))
for i in lemmata.most_common(10):
    print(i[0], i[1])

N of lemmata: 39007
в 9598
и 9005
на 4623
с 3570
по 2310
год 2245
для 1584
от 1449
что 1329
при 1182


In [49]:
lemmata_no_sw = nltk.FreqDist()
for l in lemmata:
    if not l in scientific_stops:
        lemmata_no_sw[l] = lemmata[l]
for i in lemmata_no_sw.most_common(10):
    print(i[0], i[1])

год 2245
пермский 1086
который 962
производство 888
являться 812
продукция 804
развитие 727
предприятие 681
хозяйство 674
сельскохозяйственный 651


#### journalistic

In [52]:
morph = pymorphy2.MorphAnalyzer()
lemmata = nltk.FreqDist()

j_types = nltk.FreqDist(journalistic_tokens)
for t in j_types:
    try:
        l = morph.parse(t)[0].normal_form
        if l in lemmata:
            lemmata[l] += j_types[t]
        else:
            lemmata[l] = j_types[t]
    except IndexError:
        if t in lemmata:
            lemmata[t] += j_types[t]
        else:
            lemmata[t] = j_types[t]
print('N of lemmata:', len(lemmata))
for i in lemmata.most_common(10):
    print(i[0], i[1])

N of lemmata: 4027
в 713
и 299
на 243
что 177
с 174
о 171
год 138
это 135
по 135
быть 106


In [59]:
journalistic_stops = stopwords.words('russian') + [u'это']

In [60]:
lemmata_no_sw = nltk.FreqDist()
for l in lemmata:
    if not l in journalistic_stops:
        lemmata_no_sw[l] = lemmata[l]
for i in lemmata_no_sw.most_common(20):
    print(i[0], i[1])

год 138
февраль 101
россия 84
который 78
также 51
российский 42
заявить 42
мочь 39
сообщить 38
стать 34
процент 31
время 31
страна 29
информация 29
рост 29
слово 28
сообщать 28
отметить 27
рубль 25
находиться 24


#### Подсчет

In [103]:
parts = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 
         'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 
         'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ',
         'PRCL', 'INTJ']

In [101]:
verbs = nltk.FreqDist()
for t in c_types:
    gram_info = morph.parse(t)[0]
    if 'VERB' in gram_info.tag:
        l = gram_info.normal_form
        if l in verbs:
            verbs[l] += c_types[t]
        else:
            verbs[l] = c_types[t]
print('N of verbs:', len(verbs))
for i in verbs.most_common(20):
    print( i[0], i[1])

N of verbs: 7030
быть 2555
хотеть 1117
мочь 915
любить 498
знать 482
думать 332
сказать 293
идти 275
хотеться 250
говорить 237
пойти 230
смотреть 230
сидеть 216
стать 201
писать 199
ждать 198
болеть 195
забыть 179
понимать 177
понять 176


In [105]:
def count_parts_of_speech(types):
    parts = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 
         'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 
         'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ',
         'PRCL', 'INTJ']
    for p in parts:
        words = nltk.FreqDist()
        for t in types:
            gram_info = morph.parse(t)[0]
            if p in gram_info.tag:
                l = gram_info.normal_form
                if l in words:
                    words[l] += types[t]
                else:
                    words[l] = types[t]
        print('N of words:', len(words))
        for i in words.most_common(10):
            print(i[0], i[1])

In [106]:
count_parts_of_speech(c_types)

N of words: 21192
день 855
человек 544
год 492
раз 461
время 405
дом 355
друг 309
работа 296
утро 293
жизнь 287
N of words: 3371
весь 3017
мой 1227
такой 1207
один 867
этот 853
свой 662
тот 608
самый 472
какой 401
хороший 394
N of words: 1733
должный 120
нужный 107
пришлый 87
нормальный 82
холодный 81
ладный 68
какойтый 67
обидный 66
наконецтый 60
пошлый 57
N of words: 176
большой 351
хороший 189
ранний 102
далёкий 57
маленький 49
худой 46
быстрый 37
скорый 33
близкий 31
парный 22
N of words: 7030
быть 2555
хотеть 1117
мочь 915
любить 498
знать 482
думать 332
сказать 293
идти 275
хотеться 250
говорить 237


KeyboardInterrupt: 